# **Chapter 15: Processing Sequences Using RNNs and CNNs**

## **1. Pendahuluan**

Manusia memiliki kemampuan luar biasa untuk memprediksi masa depan berdasarkan urutan peristiwa masa lalu. Saat kita mendengar kalimat, kita bisa menebak kata berikutnya. Saat kita melihat bola dilempar, kita bisa mengantisipasi lintasannya.

Dalam Machine Learning, data sekuensial (berurutan) seperti **deret waktu (time series)**, kalimat (teks), atau sinyal audio memerlukan penanganan khusus. Jaringan saraf biasa (Feedforward Neural Networks) tidak memiliki memori tentang apa yang terjadi sebelumnya. Di sinilah **Recurrent Neural Networks (RNNs)** dan **Convolutional Neural Networks (CNNs)** 1D berperan.

Bab ini membahas:
* Konsep dasar RNN dan bagaimana mereka "mengingat" informasi.
* Melatih RNN menggunakan *Backpropagation Through Time*.
* Memprediksi deret waktu (Time Series Forecasting).
* Menangani sekuens panjang menggunakan **LSTM** dan **GRU**.
* Menggunakan **1D CNN** untuk pemrosesan sekuens yang efisien.

## **2. Recurrent Neurons dan Layers**

### **Intuisi Dasar**
Bayangkan sebuah neuron yang tidak hanya menerima input dari data saat ini, tetapi juga menerima output dari dirinya sendiri pada langkah waktu sebelumnya. Ini adalah ide dasar **Recurrent Neuron**.

Pada setiap langkah waktu $t$ (time step):
* Neuron menerima input $x_{(t)}$.
* Neuron menerima output dari langkah sebelumnya $y_{(t-1)}$.

Ini menciptakan semacam memori jangka pendek. Jika kita membuka lipatan (*unroll*) proses ini terhadap waktu, RNN terlihat seperti jaringan saraf yang sangat dalam, di mana setiap layer memiliki bobot yang sama (shared weights).

**Pentingnya dalam ML:**
Kemampuan ini memungkinkan model untuk menangkap pola temporal dan dependensi antar waktu, yang sangat krusial untuk data seperti harga saham, cuaca, atau terjemahan bahasa.

## **3. Training RNN untuk Time Series Forecasting**

Salah satu aplikasi paling umum dari RNN adalah memprediksi masa depan dalam deret waktu. Sebelum masuk ke model yang kompleks, mari kita buat data dummy (synthetic time series) untuk bereksperimen.

Fungsi di bawah ini menghasilkan deret waktu yang terdiri dari dua gelombang sinus dengan frekuensi dan offset acak, ditambah sedikit noise.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # gelombang 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + gelombang 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

# Membuat dataset
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

print("Shape X_train:", X_train.shape)
print("Shape y_train:", y_train.shape)

Mari kita visualisasikan salah satu sampel data deret waktu kita.

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)

plt.figure(figsize=(10, 6))
plot_series(X_valid[0, :, 0], y_valid[0, 0])
plt.title("Contoh Time Series")
plt.show()

### **Baseline Metrics**
Sebelum menggunakan RNN, kita harus memiliki standar pembanding (*baseline*).
1.  **Naive Forecasting:** Prediksi nilai besok sama dengan nilai hari ini (nilai terakhir).
2.  **Linear Regression:** Menggunakan Fully Connected Layer sederhana.

In [ ]:
# 1. Naive Forecasting
y_pred = X_valid[:, -1]
print("Naive MSE:", np.mean(keras.losses.mean_squared_error(y_valid, y_pred)))

# 2. Linear Prediction (Simple Dense Network)
model_linear = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model_linear.compile(loss="mse", optimizer="adam")
model_linear.fit(X_train, y_train, epochs=20, verbose=0)
print("Linear Model MSE:", model_linear.evaluate(X_valid, y_valid, verbose=0))

### **Implementasi Simple RNN**
Sekarang mari kita gunakan `SimpleRNN` dari Keras. Layer ini hanya memiliki satu neuron recurrent.

**Penting:**
RNN mengharapkan input dalam bentuk 3D: `[batch_size, time_steps, dimensionality]`. Data kita sudah dalam format ini (batch, 50, 1).

In [ ]:
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model_rnn.compile(loss="mse", optimizer="adam")
history = model_rnn.fit(X_train, y_train, epochs=20, verbose=0)
print("Simple RNN MSE:", model_rnn.evaluate(X_valid, y_valid, verbose=0))

### **Deep RNNs**
Satu neuron biasanya tidak cukup. Kita bisa menumpuk beberapa layer RNN.
**Catatan Penting:** Jika kita menumpuk layer RNN, semua layer kecuali yang terakhir harus mengembalikan seluruh urutan output (`return_sequences=True`), bukan hanya output langkah terakhir.

In [ ]:
model_deep_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20), # Layer terakhir default return_sequences=False
    keras.layers.Dense(1)
])

model_deep_rnn.compile(loss="mse", optimizer="adam")
model_deep_rnn.fit(X_train, y_train, epochs=20, verbose=0)
print("Deep RNN MSE:", model_deep_rnn.evaluate(X_valid, y_valid, verbose=0))

## **4. Menangani Sekuens Panjang (Long Sequences)**

Saat melatih RNN pada sekuens yang panjang, kita menghadapi masalah:
1.  **Unstable Gradients:** Gradien bisa menghilang (*vanishing*) atau meledak (*exploding*), membuat pelatihan sulit.
2.  **Memory Loss:** Informasi dari langkah awal cenderung terlupakan saat mencapai langkah akhir.

Solusinya adalah menggunakan jenis sel yang lebih kompleks seperti **LSTM (Long Short-Term Memory)** dan **GRU (Gated Recurrent Unit)**.

### **LSTM (Long Short-Term Memory)**

LSTM memiliki "jalur tol" untuk memori jangka panjang yang memungkinkannya menyimpan informasi untuk waktu yang lama. Ia memiliki tiga gerbang (*gates*):
* **Forget Gate:** Apa yang harus dilupakan dari memori jangka panjang.
* **Input Gate:** Apa yang harus disimpan ke dalam memori.
* **Output Gate:** Apa yang harus dikeluarkan pada langkah saat ini.

### **GRU (Gated Recurrent Unit)**
Versi sederhana dari LSTM yang menggabungkan beberapa gate, namun seringkali memiliki performa yang setara dengan komputasi yang lebih ringan.

In [ ]:
# Implementasi LSTM
model_lstm = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)
])

model_lstm.compile(loss="mse", optimizer="adam")
model_lstm.fit(X_train, y_train, epochs=20, verbose=0)
print("LSTM MSE:", model_lstm.evaluate(X_valid, y_valid, verbose=0))

In [ ]:
# Implementasi GRU
model_gru = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20),
    keras.layers.Dense(1)
])

model_gru.compile(loss="mse", optimizer="adam")
model_gru.fit(X_train, y_train, epochs=20, verbose=0)
print("GRU MSE:", model_gru.evaluate(X_valid, y_valid, verbose=0))

**Interpretasi Hasil:**
Umumnya, LSTM dan GRU akan memberikan hasil MSE yang jauh lebih baik (lebih rendah) dibandingkan SimpleRNN, terutama jika pola dalam data memiliki ketergantungan jangka panjang. Mereka mampu "mengingat" tren atau siklus yang muncul jauh sebelumnya dalam urutan waktu.

## **5. Menggunakan 1D Convolutional Neural Networks**

Meskipun CNN biasanya diasosiasikan dengan gambar (2D), CNN juga sangat efektif untuk data sekuensial (1D).

**Bagaimana kerjanya?**
Filter 1D meluncur di sepanjang urutan waktu (seperti jendela geser). Ini memungkinkan model untuk mempelajari pola lokal (misalnya, lonjakan singkat dalam harga saham) di mana pun posisinya dalam urutan waktu.

**Keuntungan:**
* Sangat efisien secara komputasi (bisa diparalelkan, tidak seperti RNN yang sekuensial).
* Bisa dikombinasikan dengan RNN (misal: CNN untuk mengekstrak fitur, RNN untuk memori jangka panjang).

In [ ]:
model_cnn1d = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20),
    keras.layers.Dense(1)
])

model_cnn1d.compile(loss="mse", optimizer="adam")
model_cnn1d.fit(X_train, y_train, epochs=20, verbose=0)
print("1D CNN + GRU MSE:", model_cnn1d.evaluate(X_valid, y_valid, verbose=0))

### **WaveNet**
Salah satu arsitektur CNN 1D yang terkenal adalah **WaveNet**. Ia menggunakan *dilated convolutions* (konvolusi yang melompati input dengan jarak tertentu) yang ditumpuk. Ini memungkinkan jaringan memiliki *receptive field* yang sangat besar (melihat sejarah yang sangat panjang) dengan jumlah parameter yang sedikit.

In [ ]:
# Implementasi Sederhana WaveNet
model_wavenet = keras.models.Sequential()
model_wavenet.add(keras.layers.InputLayer(input_shape=[None, 1]))

# Stack dilated convolutions
for rate in (1, 2, 4, 8) * 2:
    model_wavenet.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                          activation="relu", dilation_rate=rate))

model_wavenet.add(keras.layers.Conv1D(filters=1, kernel_size=1))

model_wavenet.compile(loss="mse", optimizer="adam")
model_wavenet.fit(X_train, y_train, epochs=20, verbose=0)
print("WaveNet MSE:", model_wavenet.evaluate(X_valid, y_valid, verbose=0))

## **6. Kesimpulan**

Dalam Chapter 15 ini, kita telah mempelajari:
1.  **RNN** adalah alat utama untuk data sekuensial karena memiliki memori internal.
2.  **Training RNN** bisa sulit karena masalah gradien yang tidak stabil, tetapi teknik seperti Layer Normalization membantu.
3.  **LSTM dan GRU** mengatasi masalah memori jangka pendek dengan mekanisme *gating* yang cerdas, menjadi standar industri untuk banyak tugas sekuensial.
4.  **1D CNN** menawarkan alternatif yang cepat dan efisien, seringkali mampu bersaing atau melengkapi RNN, terutama dengan arsitektur seperti WaveNet.

Bab ini menutup pembahasan tentang *supervised learning* untuk data sekuensial. Di bab-bab selanjutnya (Part II buku), kita akan masuk ke topik yang lebih canggih seperti Natural Language Processing (NLP) dengan Transformer dan Generative Models.